In [1]:
import pandas as pd
from cdptools import CDPInstance, configs

In [2]:
seattle = CDPInstance(configs.SEATTLE)

In [6]:
# how to search for events
matching_events = seattle.database.search_events("bicycle infrastructure")
matching_events[0].data

In [12]:
# search for bills/appointments/minute items
matching_minutes_items = seattle.database.search_minutes_items("bicycle infrastructure")

In [26]:
matching_minutes_items[100].data

{'minutes_item_id': 'e79ee511-3bc8-4ec0-addd-8d02a41cf255',
 'matter': 'Res 31893',
 'legistar_event_item_id': 66005,
 'name': 'A RESOLUTION relating to taxation; stating an intent to adopt legislation imposing a local sales and use tax at the maximum rate authorized to fund investments in affordable and supportive housing.',
 'created': datetime.datetime(2019, 8, 8, 8, 13, 22, 265059)}

In [27]:
event_corpus_map = seattle.download_transcripts()

//anaconda3/envs/sklearn-env/lib/python3.7/site-packages/cdptools/research_utils/transcripts.py:188: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  for transcript_details in selected_transcripts.to_dict("records"):


In [29]:
event_corpus_map

316